In [105]:
# import libraries
from datetime import datetime
from pandas_datareader import data as pdr
import yfinance as yf
import pandas as pd
import time
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import random
import warnings
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [113]:
ticker = yf.Ticker("7090.KL")
# Get historical data for the ticker
hist = ticker.history(period="1y")
print(hist)

                               Open      High       Low     Close   Volume  \
Date                                                                         
2022-10-06 00:00:00+08:00  2.049363  2.049363  2.036226  2.049363    65250   
2022-10-07 00:00:00+08:00  2.049364  2.055932  2.042795  2.055932    31650   
2022-10-11 00:00:00+08:00  2.042795  2.042795  2.003384  2.009953   126300   
2022-10-12 00:00:00+08:00  2.009953  2.095342  2.009953  2.095342   124200   
2022-10-13 00:00:00+08:00  2.095342  2.095342  2.016520  2.082206   218250   
2022-10-14 00:00:00+08:00  2.042795  2.121616  2.042795  2.108480   234750   
2022-10-17 00:00:00+08:00  2.121616  2.128185  2.101911  2.108480    28050   
2022-10-18 00:00:00+08:00  2.115048  2.141322  2.108480  2.128185   130050   
2022-10-19 00:00:00+08:00  2.115048  2.147891  2.115048  2.134753    97500   
2022-10-20 00:00:00+08:00  2.115048  2.134753  2.101911  2.134753    26700   
2022-10-21 00:00:00+08:00  2.134753  2.134753  2.088774  2.10191

In [114]:
# Assuming hist is a DataFrame with the date set as the index

# Access the date of the first and last record (assuming the index is in datetime format)
# Define the purchase_date and current_date (replace these with your actual dates)
purchase_date = hist.index[0]
current_date = hist.index[-1]
purchase_date = purchase_date.strftime('%Y-%m-%d')
current_date = current_date.strftime('%Y-%m-%d')

# Access the first and last record's Close column
purchase_price = hist['Close'].iloc[0]
current_price = hist['Close'].iloc[-1]

# Now, first_record_date contains the date from the first record
print(purchase_price)


2.049363374710083


In [115]:
price_diff = current_price - purchase_price
price_perc = (current_price - purchase_price) / purchase_price * 100
print(price_perc)

18.573411467401023


In [116]:
stock_code = "7090"
df2 = pd.read_csv('historicaldividends.csv')

# Convert EX Date column to datetime
df2['EX Date'] = pd.to_datetime(df2['EX Date'])
df2['EX Date'] = pd.to_datetime(df2['EX Date'].dt.strftime('%Y-%m-%d'))

# Filter the DataFrame based on the Stock Code and EX Date range
filtered_df2 = df2[(df2['Stock Code'] == stock_code) & (df2['EX Date'] >= purchase_date) & (df2['EX Date'] <= current_date)]
filtered_df2

# Calculate the sum of the 'Amount' column in the filtered DataFrame
dividend_amount = filtered_df2['Amount'].sum()

# Print the calculated dividend_amount
print(f"Total dividend amount between {purchase_date} and {current_date} for Stock Code {stock_code}: {dividend_amount}")

Total dividend amount between 2022-10-06 and 2023-10-06 for Stock Code 7090: 0.05500000000000001


In [117]:
dividend_yield = dividend_amount / purchase_price
print(dividend_yield)

0.026837602681262264


# ---------------------------------------------------------------------------------------------------------------

In [118]:
df = pd.read_csv('exports/filtered_class_a_b.csv')
df2 = pd.read_csv('historicaldividends.csv')

# Convert EX Date column to datetime
df2['EX Date'] = pd.to_datetime(df2['EX Date'])
df2['EX Date'] = pd.to_datetime(df2['EX Date'].dt.strftime('%Y-%m-%d'))

results = []

# Loop through each ticker symbol in the CSV file
for ticker_symbol in df['Ticker']:
    # Construct the ticker object
    ticker = yf.Ticker(ticker_symbol)

    # Get historical data for the ticker
    hist = ticker.history(period="1y")

    purchase_date = hist.index[0]
    current_date = hist.index[-1]
    purchase_date = purchase_date.strftime('%Y-%m-%d')
    current_date = current_date.strftime('%Y-%m-%d')

    # Access the first and last record's Close column
    purchase_price = hist['Close'].iloc[0]
    current_price = hist['Close'].iloc[-1]

    # Now, first_record_date contains the date from the first record
    print(purchase_price)
    
    price_diff = current_price - purchase_price
    price_perc = (current_price - purchase_price) / purchase_price * 100
    print(price_perc)
    
    # Filter the DataFrame based on the Stock Code and EX Date range
    filtered_df2 = df2[(df2['Stock Code'] == ticker_symbol[:-3]) & (df2['EX Date'] >= purchase_date) & (df2['EX Date'] <= current_date)]
    filtered_df2

    # Calculate the sum of the 'Amount' column in the filtered DataFrame
    dividend_amount = filtered_df2['Amount'].sum()

    # Print the calculated dividend_amount
    print(f"Total dividend amount between {purchase_date} and {current_date} for Stock Code {stock_code}: {dividend_amount}")
    
    dividend_yield = dividend_amount / purchase_price
    print(dividend_yield)
    
    # Append the results to the list for this ticker symbol
    results.append({
        'Ticker': ticker_symbol,
        'Purchase Date': purchase_date,
        'Current Date': current_date,
        'Purchase Price': purchase_price,
        'Current Price': current_price,
        'Price Difference': price_diff,
        'Price Percentage Change': price_perc,
        'Dividend Amount': dividend_amount,
        'Dividend Yield': dividend_yield
    })

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

# Now, you have the backtesting results for each ticker in the CSV file


2.049363374710083
18.573411467401023
Total dividend amount between 2022-10-06 and 2023-10-06 for Stock Code 7090: 0.05500000000000001
0.026837602681262264
1.7610386610031128
40.8259837782802
Total dividend amount between 2022-10-06 and 2023-10-06 for Stock Code 7090: 0.22
0.12492627497154711
1.0300841331481934
12.612157521167758
Total dividend amount between 2022-10-06 and 2023-10-06 for Stock Code 7090: 0.055
0.05339369691280101
3.1552674770355225
13.144129910562452
Total dividend amount between 2022-10-06 and 2023-10-06 for Stock Code 7090: 0.135
0.042785596144399446


In [119]:
results_df

,Ticker,Purchase Date,Current Date,Purchase Price,Current Price,Price Difference,Price Percentage Change,Dividend Amount,Dividend Yield
0,7090.KL,2022-10-06,2023-10-06,2.049363,2.43,0.380637,18.573411,0.055,0.026838
1,5248.KL,2022-10-06,2023-10-06,1.761039,2.48,0.718961,40.825984,0.220,0.124926
2,2062.KL,2022-10-06,2023-10-06,1.030084,1.16,0.129916,12.612158,0.055,0.053394
3,6139.KL,2022-10-06,2023-10-06,3.155267,3.57,0.414732,13.144130,0.135,0.042786
